In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.assignment2.simulation import HospitalSimulation, PatientState, WardType, WardConfig, WardsConfigurations, HospitalConfiguration
from scipy import stats
import numpy as np


In [3]:

P = np.zeros([6,6])
P[0, :] = [0.0, 0.05, 0.10, 0.05, 0.80, 0.0]
P[1, :] = [0.20, 0.0, 0.50, 0.15, 0.15, 0.0]
P[2, :] = [0.30, 0.20, 0.0, 0.20, 0.30, 0.0]
P[3, :] = [0.35, 0.30, 0.05, 0.0, 0.30, 0.0]
P[4, :] = [0.20, 0.10, 0.60, 0.10, 0.0, 0.0] 
P[5, :] = [0.20, 0.20, 0.20, 0.20, 0.20, 0.0]

arr_Times = [14.5, 11.0, 8.0, 6.5, 5.0, 13.0]

len_stay = [2.9, 4.0, 4.5, 1.4, 3.9, 2.2]

urgency = [7, 5, 2, 10, 5, 0]

bed_capacity = [55, 40, 30, 20, 20, 0]


In [4]:

configs = [
    WardConfig(bed_capacity=55, urgency=7, mean_arrival_time=1/14.5, 
        mean_stay_time=2.9),
    WardConfig(40, 5, 1/11, 4),
    WardConfig(30, 2, 1/8, 4.5),
    WardConfig(20, 10, 1/6.5, 1.4),
    WardConfig(20, 5, 1/5, 3.9),
    WardConfig(0, 0, 1/13, 2.2)
]

configs_nof = configs.copy()
del configs_nof[-1]

std_configs = WardsConfigurations(configs)
std_switch = {
    WardType.A: [0.0, 0.05, 0.10, 0.05, 0.80, 0.0],
    WardType.B: [0.20, 0.0, 0.50, 0.15, 0.15, 0.0],
    WardType.C: [0.30, 0.20, 0.0, 0.20, 0.30, 0.0],
    WardType.D: [0.35, 0.30, 0.05, 0.0, 0.30, 0.0],
    WardType.E: [0.20, 0.10, 0.60, 0.10, 0.0, 0.0],
    WardType.F: [0.20, 0.20, 0.20, 0.20, 0.20, 0.0]
}

def mean_exp_dist(mean_time):
    return stats.expon.rvs(scale=mean_time)

sim_config = HospitalConfiguration(std_configs, std_switch, mean_exp_dist, mean_exp_dist)
hsim = HospitalSimulation(sim_config)

In [ ]:
hsim.simulate_year(display=False)
hsim.total_penalty

In [47]:
hsim.wards[WardType.F].rejected_fraction

0.04982499485278979

In [6]:
## Example of setting bed distribution, individually and alltogether
hsim.ward_configs[WardType.A].bed_capacity = 40
hsim.ward_configs.bed_distribution = [100, 40, 10, 10, 10, 0]

## Example setting new distributions
def mean_lognorm_sd2(mean_time):
    s = 2/mean_time**2
    mu = mean_time * np.exp(-s**2/2)
    return stats.lognorm(scale=mu, s=np.sqrt(s)).rvs()

hsim.stay_dist = mean_lognorm_sd2


In [10]:
hsim.simulate_year()

In [12]:
hsim.wards[WardType.E].rejected_fraction

0.9308980213089803

In [11]:
nof_sim.ward_configs

[WardConfig(bed_capacity=100, urgency=7, mean_arrival_time=0.06896551724137931, mean_stay_time=2.9),
 WardConfig(bed_capacity=40, urgency=5, mean_arrival_time=0.09090909090909091, mean_stay_time=4),
 WardConfig(bed_capacity=10, urgency=2, mean_arrival_time=0.125, mean_stay_time=4.5),
 WardConfig(bed_capacity=10, urgency=10, mean_arrival_time=0.15384615384615385, mean_stay_time=1.4),
 WardConfig(bed_capacity=10, urgency=5, mean_arrival_time=0.2, mean_stay_time=3.9)]